<a href="https://colab.research.google.com/github/Squirtle007/CUDA-Q/blob/main/02_noisy_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/Squirtle007/CUDA_Quantum/blob/main/colab/v0.8.0/cudaq_tutorial_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set up CUDA-Q Environment**

In [1]:
#The following commands are used to set up the environment in Colab
%pip install cuda-quantum==0.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.8/148.8 MB 7.7 MB/s eta 0:00:00


## Noisy Simulation

Quantum noise can be characterized into coherent and incoherent sources of errors that arise during a computation. Coherent noise is commonly due to systematic errors originating from device miscalibrations, for example, gates implementing a rotation $\theta + \epsilon$ instead of $\theta$.

Incoherent noise has its origins in quantum states being entangled with the environment due to decoherence. This leads to mixed states which are probability distributions over pure states and are described by employing the density matrix formalism.

We can model incoherent noise via quantum channels which are linear, completely positive, and trace preserving maps. These maps are called Kraus operators, $\{ K_i \}$, which satisfy the condition $\sum_{i} K_i^\dagger K_i = \mathbb{I}$.

The bit-flip channel flips the qubit with probability $p$ and leaves it unchanged with probability $1-p$. This can be represented by employing Kraus operators:

$$K_0 = \sqrt{1-p} \begin{pmatrix}
    1 & 0 \\
    0 & 1
\end{pmatrix}$$

$$K_1 = \sqrt{p} \begin{pmatrix}
  0 & 1 \\
  1 & 0
\end{pmatrix}$$

Let's implement the **bit-flip** channel as well as **depolarization** channel using CUDA-Q:

In [2]:
import cudaq
from cudaq import spin

import numpy as np

# To model quantum noise, we need to utilize the density matrix simulator target.
cudaq.set_target("density-matrix-cpu")

In [3]:
# Let's define a simple kernel that we will add noise to.
qubit_count = 2


@cudaq.kernel
def kernel(qubit_count: int):
    qvector = cudaq.qvector(qubit_count)
    x(qvector)


print(cudaq.draw(kernel, qubit_count))

     ╭───╮
q0 : ┤ x ├
     ├───┤
q1 : ┤ x ├
     ╰───╯



In [4]:
# In the ideal noiseless case, we get |11> 100% of the time.

ideal_counts = cudaq.sample(kernel, qubit_count, shots_count=1000)
print(ideal_counts)  #ideal_counts.dump()

{ 11:1000 }



In [5]:
# First, we will define an out of the box noise channel. In this case,
# we choose depolarization noise. This depolarization will result in
# the qubit state decaying into a mix of the basis states, |0> and |1>,
# with our provided probability.
error_probability = 0.1
depolarization_channel = cudaq.DepolarizationChannel(error_probability)

# We can also define our own, custom noise channels through
# Kraus operators. Here we will define two operators representing
# bit flip errors.

# Define the Kraus Error Operator as a complex ndarray.
kraus_0 = np.sqrt(1 - error_probability) * np.array([[1.0, 0.0], [0.0, 1.0]],
                                                    dtype=np.complex128)
kraus_1 = np.sqrt(error_probability) * np.array([[0.0, 1.0], [1.0, 0.0]],
                                                dtype=np.complex128)

# Add the Kraus Operator to create a quantum channel.
bitflip_channel = cudaq.KrausChannel([kraus_0, kraus_1])

# Add the two channels to our Noise Model.
noise_model = cudaq.NoiseModel()

# Apply the depolarization channel to any X-gate on the 0th qubit.
noise_model.add_channel("x", [0], depolarization_channel)
# Apply the bitflip channel to any X-gate on the 1st qubit.
noise_model.add_channel("x", [1], bitflip_channel)

# Due to the impact of noise, our measurements will no longer be uniformly
# in the |11> state.
noisy_counts = cudaq.sample(kernel,
                            qubit_count,
                            noise_model=noise_model,
                            shots_count=1000)

print(noisy_counts)  #noisy_counts.dump()

{ 11:846 10:82 01:63 00:9 }



In [6]:
# We can also use noise models with the observe function

hamiltonian = spin.z(0)

noisy_result = cudaq.observe(kernel,
                             hamiltonian,
                             qubit_count,
                             noise_model=noise_model)

noisy_result.expectation()

-0.8666666666666666

Then implement the **phase-flip** channel using CUDA-Q:

In [7]:
import cudaq

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# CUDA-Q supports several different models of noise. In this
# case, we will examine the modeling of decoherence of the qubit phase.
# This will occur from "phase flip" errors, wherein the qubit has a
# user-specified probability of undergoing a Z-180 rotation.

# We will begin by defining an empty noise model that we will add
# our phase flip channel to.
noise = cudaq.NoiseModel()

# We define a phase-flip channel setting to `1.0` the probability of the qubit
# undergoing a phase rotation of 180 degrees (π).
phase_flip = cudaq.PhaseFlipChannel(1.0)
# We will apply this channel to any Z gate on the qubit.
# In other words, after each Z gate on qubit 0, there will be a
# probability of `1.0` that the qubit undergoes an extra
# Z rotation.
noise.add_channel('z', [0], phase_flip)


@cudaq.kernel
def kernel():
    # Single qubit initialized to the |0> state.
    qubit = cudaq.qubit()
    # Place qubit in superposition state.
    h(qubit)
    # Rotate the phase around Z by 180 degrees (π).
    z(qubit)
    # Apply another Hadamard and measure.
    h(qubit)
    mz(qubit)


print(cudaq.draw(kernel))

     ╭───╮╭───╮╭───╮
q0 : ┤ h ├┤ z ├┤ h ├
     ╰───╯╰───╯╰───╯



In [8]:
# Without noise, we'd expect the qubit to end in the |1>
# state due to the phase rotation between the two Hadamard
# gates.
noiseless_result = cudaq.sample(kernel)
print(noiseless_result)

{ 1:1000 }



In [9]:
# With noise, our Z-gate will effectively cancel out due
# to the presence of a phase flip error on the gate with a
# probability of `1.0`. This will put us back in the |0> state.
noisy_result = cudaq.sample(kernel, noise_model=noise)
print(noisy_result)

{ 0:1000 }

